In [35]:
import os
import pandas as pd
import torch
from transformers import BertTokenizer
import random

In [36]:
PRETRAINED_MODEL_NAME = "bert-base-cased"  # 指定英文的 BERT-BASE 預訓練模型
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

I1016 17:12:11.381353  4948 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\Grace\.cache\torch\transformers\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


In [37]:
# 簡單的數據清理，去除空白標題的 examples
df_train = pd.read_csv("task1_trainset.csv")

empty_title = ((df_train['Authors'].isnull()) \
			   | (df_train['Categories'].isnull()) \
			   | (df_train['Created Date'].isnull()))
df_train = df_train[~empty_title]

In [4]:
df_train[:10]

,Id,Title,Abstract,Authors,Categories,Created Date,Task 1
0,D00001,A Brain-Inspired Trust Management Model to Ass...,Rapid popularity of Internet of Things (IoT) a...,Mahmud/Kaiser/Rahman/Rahman/Shabut/Al-Mamun/Hu...,cs.CR/cs.AI/q-bio.NC,2018-01-11,BACKGROUND OBJECTIVES METHODS METHODS RESULTS ...
1,D00002,On Efficient Computation of Shortest Dubins Pa...,"In this paper, we address the problem of compu...",Sadeghi/Smith,cs.SY/cs.RO/math.OC,2016-09-21,OBJECTIVES OTHERS METHODS/RESULTS RESULTS RESULTS
2,D00003,Data-driven Upsampling of Point Clouds,High quality upsampling of sparse 3D point clo...,Zhang/Jiang/Yang/Yamakawa/Shimada/Kara,cs.CV,2018-07-07,BACKGROUND OBJECTIVES METHODS METHODS METHODS ...
3,D00004,Accessibility or Usability of InteractSE? A He...,Internet is the main source of information now...,Aqle/Khowaja/Al-Thani,cs.HC,2018-08-29,BACKGROUND BACKGROUND BACKGROUND OBJECTIVES OB...
4,D00005,Spatio-Temporal Facial Expression Recognition ...,Automated Facial Expression Recognition (FER) ...,Hasani/Mahoor,cs.CV,2017-03-20,BACKGROUND BACKGROUND BACKGROUND BACKGROUND ME...
5,D00006,Continuous Semantic Topic Embedding Model Usin...,This paper proposes the continuous semantic to...,Jung/Choi,stat.ML/cs.CL/cs.LG,2017-11-24,OBJECTIVES/METHODS METHODS CONCLUSIONS RESULTS...
6,D00007,Beyond Shared Hierarchies: Deep Multitask Lear...,Existing deep multitask learning (MTL) approac...,Meyerson/Miikkulainen,cs.LG/cs.AI/stat.ML,2017-10-31,BACKGROUND BACKGROUND/OBJECTIVES OBJECTIVES/ME...
7,D00008,Using Scalp Electrical Biosignals to Control a...,In this paper we explore the use of electrical...,George/Lotte/Abad/Lécuyer,cs.OH,2011-11-08,OBJECTIVES METHODS METHODS METHODS METHODS RES...
8,D00009,Robustness from structure: Inference with hier...,How spiking networks are able to perform proba...,Petrovici/Schroeder/Breitwieser/Grübl/Schemmel...,q-bio.NC/cs.NE/stat.ML,2017-03-12,BACKGROUND/OBJECTIVES BACKGROUND BACKGROUND/ME...
9,D00010,Statistical Mechanical Analysis of Low-Density...,Low-density parity-check (LDPC) codes on symme...,Mori/Tanaka,cs.IT/math.IT,2011-10-10,BACKGROUND OBJECTIVES RESULTS/CONCLUSIONS METHODS


In [38]:
# 去除不必要的欄位並重新命名兩標題的欄位名
df_train = df_train.reset_index()
df_train = df_train.loc[:, ['Abstract', 'Task 1']]

In [39]:
Abstract_snt_list = []
label_snt_list = []
for data in df_train['Abstract']:
    data = data.split('$$$')
    #一句一句話分離
    for snt in data:
        Abstract_snt_list.append(snt)
for label in df_train['Task 1']:
    label = label.split(" ")
    for la in label:
        label_snt_list.append(la.split("/"))

In [83]:
Abstract_snt_list[:10]

['Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.',
 'To ensure secure and reliable data communication between end-to-end (E2E) devices supported by current IoT and cloud infrastructure, trust management is needed at the IoT and user ends.',
 'This paper introduces a Neuro-Fuzzy based Brain-inspired trust management model (TMM) to secure IoT devices and relay nodes, and to ensure data reliability.',
 'The proposed TMM utilizes node behavioral trust and data trust estimated using Adaptive Neuro-Fuzzy Inference System and weighted-additive methods respectively to assess the nodes trustworthiness.',
 'In contrast to the existing fuzzy based TMMs, the NS2 simulation results confirm the robustness and accuracy of the proposed TMM in identifying malicious nodes in the communication network.',
 'With the growing usage of clo

In [84]:
label_snt_list[:10]

[['BACKGROUND'],
 ['OBJECTIVES'],
 ['METHODS'],
 ['METHODS'],
 ['RESULTS'],
 ['CONCLUSIONS'],
 ['OBJECTIVES'],
 ['OTHERS'],
 ['METHODS', 'RESULTS'],
 ['RESULTS']]

In [40]:
abstract_dict = {
    'text': Abstract_snt_list,
    'label': label_snt_list
}

In [41]:
X = pd.DataFrame(abstract_dict)

In [87]:
X[:10]

,text,label
0,Rapid popularity of Internet of Things (IoT) a...,[BACKGROUND]
1,To ensure secure and reliable data communicati...,[OBJECTIVES]
2,This paper introduces a Neuro-Fuzzy based Brai...,[METHODS]
3,The proposed TMM utilizes node behavioral trus...,[METHODS]
4,"In contrast to the existing fuzzy based TMMs, ...",[RESULTS]
5,With the growing usage of cloud based IoT fram...,[CONCLUSIONS]
6,"In this paper, we address the problem of compu...",[OBJECTIVES]
7,Given initial and final configurations of the ...,[OTHERS]
8,We provide a novel geometrical analysis of the...,"[METHODS, RESULTS]"
9,We then show how our method can be used to qui...,[RESULTS]


In [142]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data_le = pd.DataFrame(abstract_dict)
data_le['label'] = labelencoder.fit_transform(X['label'])
data_le[:10]

,text,label
0,Rapid popularity of Internet of Things (IoT) a...,0
1,To ensure secure and reliable data communicati...,21
2,This paper introduces a Neuro-Fuzzy based Brai...,17
3,The proposed TMM utilizes node behavioral trus...,17
4,"In contrast to the existing fuzzy based TMMs, ...",30
5,With the growing usage of cloud based IoT fram...,16
6,"In this paper, we address the problem of compu...",21
7,Given initial and final configurations of the ...,29
8,We provide a novel geometrical analysis of the...,19
9,We then show how our method can be used to qui...,30


In [42]:
# Load libraries
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

In [10]:
X.columns

Index(['text', 'label'], dtype='object')

In [15]:
# Create MultiLabelBinarizer object
one_hot = MultiLabelBinarizer()
#data_one_hot = pd.DataFrame(abstract_dict)
#data_one_hot.label = one_hot.fit_transform(X.label)
# One-hot encode data
X_one_hot = one_hot.fit(['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVES', 'OTHERS',
       'RESULTS'])
#test = X_one_hot.transform(X.label)
#X_one_hot = one_hot.fit_transform(X.label)
'''
s = X['label']
one_hot = MultiLabelBinarizer()
X_one_hot = pd.DataFrame(one_hot.fit_transform(s),columns=one_hot.classes_, index=X.index)
'''
# View classes
one_hot.classes_

array(['A', 'B', 'C', 'D', 'E', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N',
       'O', 'R', 'S', 'T', 'U', 'V'], dtype=object)

In [43]:
data_one_hot = X.label.str.join('|').str.get_dummies().add_prefix('')
df = pd.concat([X,data_one_hot],axis=1).drop(['label'],axis=1)

In [30]:
test = df.loc[:10,['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVES', 'OTHERS','RESULTS']]

In [71]:
df.loc[8,['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVES', 'OTHERS','RESULTS']].values

array([0, 0, 1, 0, 0, 1], dtype=object)

In [52]:
list(df.loc[8,['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVES', 'OTHERS','RESULTS']])

[0, 0, 1, 0, 0, 1]

In [143]:
# test.csv的處理
df_test = pd.read_csv("task1_public_testset.csv")
df_test = df_test.loc[:, ['Abstract', "Id"]]
df_test.columns = ["text", "Id"]

In [145]:
for data in df_test["text"]:
    print(df_test.loc[df_test['text'].index(data),'Id'])

TypeError: 'RangeIndex' object is not callable

In [85]:
import torch
from torch import nn
from transformers import BertTokenizer
import random

PRETRAINED_MODEL_NAME = "bert-base-cased"  # 指定英文的 BERT-BASE 預訓練模型
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)

###----1. 準備原始文本數據

def data_preprocessing(csv_name,mode):
	df = pd.read_csv(csv_name)
	empty_title = ((df['Authors'].isnull()) \
			   | (df['Categories'].isnull()) \
			   | (df['Created Date'].isnull()))
	df = df[~empty_title]

	#重設columns
	df = df.reset_index()
	if mode=='train':
		df = df.loc[:, ['Abstract', 'Task 1']]
		df.columns = ['text', 'label']
	else:
		df_test = df_test.loc[:, ['Abstract', "Id"]]
		df_test.columns = ["text", "Id"]

	#將abstract內每句話和label分離
	Abstract_snt_list = []
	label_snt_list = []
	id_snt_list = []
	for data in df['text']:
		data = data.split('$$$')
		#一句一句話分離
		for snt in data:
			Abstract_snt_list.append(snt)
		'''
		#還沒改好test部分
		if mode=='test':
			id_snt_list.append()
			abstract_dict = {
				'text': Abstract_snt_list,
				'id': label_snt_list
			}
			'''
	if mode=='train':
		for label in df['label']:
			label = label.split(" ")
			for la in label:
				label_snt_list.append(la.split("/"))
		abstract_dict = {
		'text': Abstract_snt_list,
		'label': label_snt_list
		}

	return abstract_dict

def one_hot_encoding(X):	#對多標籤進行one hot encoding
	data_one_hot = X.label.str.join('|').str.get_dummies().add_prefix('')	#one hot encoding 後將結果併入原dataframe
	df = pd.concat([X,data_one_hot],axis=1).drop(['label'],axis=1)	#去除未轉換前的label
	return df


#轉成dataframe並進行ont hot encoding後存起來
abstract_dict = data_preprocessing('task1_trainset.csv','train')
df_train = pd.DataFrame(abstract_dict)
df_train = one_hot_encoding(df_train)
df_train.to_csv("task1_trainset.tsv", sep="\t", index=False)

print("訓練樣本數：", len(df_train))
print(df_train.columns)
print(df_train.head())

I1016 18:18:31.422483  4948 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-vocab.txt from cache at C:\Users\Grace\.cache\torch\transformers\5e8a2b4893d13790ed4150ca1906be5f7a03d6c4ddf62296c383f6db42814db2.e13dbb970cb325137104fb2e5f36fe865f27746c6b526f6352861b1980eb80b1


訓練樣本數： 46867
Index(['text', 'BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVES', 'OTHERS',
       'RESULTS'],
      dtype='object')
                                                text  BACKGROUND  CONCLUSIONS  \
0  Rapid popularity of Internet of Things (IoT) a...           1            0   
1  To ensure secure and reliable data communicati...           0            0   
2  This paper introduces a Neuro-Fuzzy based Brai...           0            0   
3  The proposed TMM utilizes node behavioral trus...           0            0   
4  In contrast to the existing fuzzy based TMMs, ...           0            0   

   METHODS  OBJECTIVES  OTHERS  RESULTS  
0        0           0       0        0  
1        0           1       0        0  
2        1           0       0        0  
3        1           0       0        0  
4        0           0       0        1  


In [98]:
from torch.utils.data import Dataset
import pysnooper
 
	
class PaperDataset(Dataset):
	# 讀取前處理後的 tsv 檔並初始化一些參數
	def __init__(self, mode, tokenizer):
		assert mode in ["task1_trainset", "task1_public_testset"]  # 一般訓練你會需要 dev set
		self.mode = mode
		# 大數據你會需要用 iterator=True
		self.df = pd.read_csv(mode + ".tsv", sep="\t").fillna("")
		self.len = len(self.df)
		#self.label_map = {'BACKGROUND': 0, 'OBJECTIVES': 1, 'METHODS': 2, 'RESULTS': 3, 'CONCLUSIONS': 4, 'OTHERS': 5}
		self.tokenizer = tokenizer  # 我們將使用 BERT tokenizer
	# 定義回傳一筆訓練 / 測試數據的函式
	def __getitem__(self, idx):	#Called to implement evaluation of self[key] (special method in python)
		if self.mode == "task1_public_testset":
			text = self.df.iloc[idx, :1].values #把dataframe每行資料的前兩個col取出，也就是把text_a和text_b單獨取出來
			label_tensor = None
		else:
			text= self.df.iloc[idx, :1].values
			label_id = list(self.df.loc[idx,['BACKGROUND', 'CONCLUSIONS', 'METHODS', 'OBJECTIVES', 'OTHERS','RESULTS']])
			label_tensor = torch.FloatTensor(label_id)
		
		#建立句子的 BERT tokens 並加入分隔符號 [SEP]
		# 建立第一個(Title)句子的 BERT tokens 並加入分隔符號 [SEP]
		word_pieces = ["[CLS]"]
		tokens = self.tokenizer.tokenize(str(text[0]))
		word_pieces += tokens + ["[SEP]"]
		len_tokens = len(word_pieces)

		# 將整個 token 序列轉換成索引序列
		ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
		tokens_tensor = torch.tensor(ids)
		
		# 將abstract包含 [SEP] 的 token 位置設為 1, 0 為pedding
		segments_tensor = torch.tensor([0] * len_tokens,
										dtype=torch.long)
		
		return (tokens_tensor, segments_tensor, label_tensor)
	
	def __len__(self):
		return self.len
	

In [51]:
text = df.iloc[0, :1].values

In [58]:
str(text[0])

'Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.'

In [59]:
tokenizer.tokenize(str(text[0]))

['Rapid',
 'popularity',
 'of',
 'Internet',
 'of',
 'Things',
 '(',
 'I',
 '##o',
 '##T',
 ')',
 'and',
 'cloud',
 'computing',
 'permits',
 'ne',
 '##uro',
 '##s',
 '##cie',
 '##nti',
 '##sts',
 'to',
 'collect',
 'multi',
 '##lev',
 '##el',
 'and',
 'multi',
 '##chan',
 '##nel',
 'brain',
 'data',
 'to',
 'better',
 'understand',
 'brain',
 'functions',
 ',',
 'di',
 '##ag',
 '##nose',
 'diseases',
 ',',
 'and',
 'de',
 '##vise',
 'treatments',
 '.']

In [99]:
	
# 初始化一個專門讀取訓練樣本的 Dataset，使用英文 BERT 斷詞
trainset = PaperDataset("task1_trainset", tokenizer=tokenizer)

In [100]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
abstract = trainset.df['text'][sample_idx]
label = trainset.df.iloc[sample_idx,7:].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = " ".join(tokens)

# 渲染前後差異，毫無反應就是個 print。可以直接看輸出結果
print(f"""[原始文本]
句子 1：{abstract}
分類  ：{label}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{label_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子 1：Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.
分類  ：[]

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([  101, 16356,  5587,  1104,  4639,  1104,  7149,   113,   146,  1186,
         1942,   114,  1105,  7180, 12783, 15267, 24928, 11955,  1116, 25982,
        14964, 10047,  1106,  7822,  4321, 23403,  1883,  1105,  4321, 18546,
         8967,  3575,  2233,  1106,  1618,  2437,  3575,  4226,   117,  4267,
         8517, 22583,  8131,   117,  1105,  1260, 16641, 14115,   119,   102])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])

label_tensor   ：tensor([1., 0., 0., 0., 0., 0.])

--------------------

[還原 tokens_tensors]
[CLS] Rapid popularity o

In [15]:
trainset.df.iloc[0].values

array(['Rapid popularity of Internet of Things (IoT) and cloud computing permits neuroscientists to collect multilevel and multichannel brain data to better understand brain functions, diagnose diseases, and devise treatments.',
       1, 0, 0, 0, 0, 0], dtype=object)

In [101]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `PaperDataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors
def create_mini_batch(samples):
	tokens_tensors = [s[0] for s in samples]
	segments_tensors = [s[1] for s in samples]
	
	# 取第一個element 內看看它的label_tensor是否為None，若為None代表是測試集
	if samples[0][2] is not None:
		#torch.stack: 沿着一个新维度对输入张量序列进行连接。 序列中所有的张量都应该为相同形状。
		label_ids = torch.stack([s[2] for s in samples]) # 將m1和m2兩個矩陣在新的維度（第一維）疊起來
		#label_ids = [s[2] for s in samples]
	else:
		label_ids = None
	
	# zero pad 到同一序列長度
	tokens_tensors = pad_sequence(tokens_tensors, 
								  batch_first=True)
	segments_tensors = pad_sequence(segments_tensors, 
									batch_first=True)
	
	# attention masks，將 tokens_tensors 裡頭不為 zero padding
	# 的位置設為 1 讓 BERT 只關注這些位置的 tokens
	masks_tensors = torch.zeros(tokens_tensors.shape, 
								dtype=torch.long)
	masks_tensors = masks_tensors.masked_fill(
		tokens_tensors != 0, 1)
	
	return tokens_tensors, segments_tensors, masks_tensors, label_ids


# 初始化一個每次回傳 64 個訓練樣本的 DataLoader﹝因為不可能一次把整個數據集放入GPU﹞
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
# `collate_fn` merges a list of samples to form a mini-batch of Tensor(s). Used when using batched loading from a map-style dataset.
# `batch_size`: how many samples per batch to load (default: 1).
BATCH_SIZE = 64
# DataLoader combines a dataset and a sampler, and provides an iterable over the given dataset.
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
						 collate_fn=create_mini_batch)

In [102]:
from torch.utils.data import Dataset

data = next(iter(trainloader))

tokens_tensors, segments_tensors, \
    masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([64, 86]) 
tensor([[  101, 16356,  5587,  ...,     0,     0,     0],
        [  101,  1706,  4989,  ...,     0,     0,     0],
        [  101,  1188,  2526,  ...,     0,     0,     0],
        ...,
        [  101,  1109,  1648,  ...,     0,     0,     0],
        [  101, 16005,  1437,  ...,     0,     0,     0],
        [  101, 12859, 20913,  ...,     0,     0,     0]])
------------------------
segments_tensors.shape = torch.Size([64, 86])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([64, 86])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------

In [131]:
###---3.  在 BERT 之上加入新 layer 成下游任務模型
# 載入一個可以做英文多分類任務的模型，n_class = 6
from transformers import BertForSequenceClassification
from transformers.modeling_bert import BertPreTrainedModel, BertModel

PRETRAINED_MODEL_NAME = "bert-base-cased"
NUM_LABELS = 6

class BertForSequenceClassification(BertPreTrainedModel):
	def __init__(self, config):
		super(BertForSequenceClassification, self).__init__(config)
		self.num_labels = config.num_labels

		self.bert = BertModel(config)  # 載入預訓練 BERT
		self.dropout = nn.Dropout(config.hidden_dropout_prob)
		# 簡單 linear 層
		self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

		self.init_weights()

	def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None):
		# BERT 輸入就是 tokens, segments, masks
		outputs = self.bert(input_ids, token_type_ids, attention_mask)
		#_,pooled_output = self.bert(input_ids, token_type_ids, attention_mask)
		# 線性分類器將 dropout 後的 BERT repr. 轉成類別 logits
		pooled_output = outputs[1]

		pooled_output = self.dropout(pooled_output)
		# 線性分類器將 dropout 後的 BERT repr. 轉成類別 logits
		logits = self.classifier(pooled_output)
		logits = logits.sigmoid()

		# 輸入有 labels 的話直接計算 Cross Entropy 回傳，方便！
		if labels is not None:
			#  We are doing regression
			#注意对于BCELoss必须保证其输入的参数都位于0到1之间，无论是标签还是预测值，
			#如果不想让预测值必须位于0到1之间则可以采用BCEWithLogitsLoss损失函数。
			loss_fct = torch.nn.BCEWithLogitsLoss()
			#loss_fct = torch.nn.BCELoss()
			#loss = loss_fct(m(logits.view(-1)), labels.view(-1))
			loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1, self.num_labels))
			return loss
		# 回傳各類別的 logits
		return logits

model = BertForSequenceClassification.from_pretrained(
	PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)


# high-level 顯示此模型裡的 modules
print("""
name            module
----------------------""")
for name, module in model.named_children():
	if name == "bert":
		for n, _ in module.named_children():
			print(f"{name}:{n}")
	else:
		print("{:15} {}".format(name, module))

I1016 20:16:42.064945  4948 configuration_utils.py:151] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-config.json from cache at C:\Users\Grace\.cache\torch\transformers\b945b69218e98b3e2c95acf911789741307dec43c698d35fad11c1ae28bda352.d7a3af18ce3a2ab7c0f48f04dc8daff45ed9a3ed333b9e9a79d012a0dedf87a6
I1016 20:16:42.068900  4948 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 6,
  "output_attentions": false,
  "output_hidden_states": false,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 28996
}

I1016 20:16:42.948754  4948 modeling_utils.py:337] loading wei


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=6, bias=True)


In [132]:
def get_predictions(model, dataloader, compute_acc=False):
	predictions = None
	correct = 0
	total = 0
	threshold = 0.5
	model.eval()  # 推論模式
	with torch.no_grad():
		# 遍巡整個資料集
		for data in dataloader:
			# 將所有 tensors 移到 GPU 上
			if next(model.parameters()).is_cuda:
				data = [t.to("cuda:0") for t in data if t is not None]
				#data是三類tensor組合而成的list，最前面都是tokens_tensor，接著是segment_tensor，最後是label_tensor
			
			#print("data:")
			#print(data)	
			#print("*data[:3]:")
			#print(*data[:3])	            
			outputs = model(*data[:3])	#*表示接受任意多个参数并将其放在一个元组中，每種tensor算一個element，故data有3個index
			#outputs為每筆資料預測的機率
			#print("outputs:")
			#print(outputs)

			# 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
			#logits = outputs[0]
			logits = outputs          
			print(logits)            
			#pred = torch.sigmoid(logits.data)
			#由于每一个样本可能属于多个类别，因此不能直接根据argmax(output,1)去直接取概率最大的输出值作为其类别，
			#而是应该设置一个阈值（默认0.5）来对output进行过滤，得到样本所属的multi-class。
			for item in logits:
				for i in range(len(item)):	#傳回來的logits若高於threshold則代表有該標籤，設為1
					if item[i] >= threshold:
						item[i] = 1
			pred = logits
			#print(pred)
			# 用來計算訓練集的分類準確率
			if compute_acc:
				labels = data[3]	#label放在最後一個
				print("labels:")
				print(labels)
				total += labels.size(0)*6	#會印出64，因為每個batch取64個documents
				print("total="+str(total))
				correct += (pred == labels).sum().item()
				print(correct)
			# 將當前 batch 記錄下來
			if predictions is None:
				predictions = pred
			else:
				predictions = torch.cat((predictions, pred))
			print('predictions:')
			print(predictions)
			break
	
	if compute_acc:
		acc = correct / total
		return predictions, acc
	return predictions
	
# 讓模型跑在 GPU 上並取得訓練集的分類準確率﹝初始化狀態的測試﹞
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)	#Performs Tensor dtype and/or device conversion
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cpu
tensor([[0.5847, 0.3990, 0.5506, 0.4000, 0.4802, 0.5224],
        [0.6458, 0.4610, 0.5421, 0.3557, 0.4431, 0.5530],
        [0.6184, 0.4146, 0.5475, 0.3818, 0.4435, 0.5472],
        [0.5930, 0.4135, 0.5404, 0.4055, 0.4662, 0.5686],
        [0.6176, 0.4275, 0.5462, 0.3710, 0.4514, 0.5697],
        [0.6230, 0.4522, 0.5551, 0.3660, 0.4584, 0.5618],
        [0.5799, 0.4119, 0.5684, 0.4185, 0.4583, 0.5401],
        [0.6268, 0.4888, 0.5577, 0.3530, 0.4369, 0.5666],
        [0.6092, 0.4498, 0.5796, 0.3761, 0.4647, 0.5659],
        [0.6202, 0.4320, 0.5509, 0.3720, 0.4426, 0.5638],
        [0.6411, 0.4709, 0.5719, 0.3537, 0.4367, 0.5702],
        [0.6271, 0.4443, 0.5626, 0.3875, 0.4419, 0.5288],
        [0.6032, 0.4258, 0.5757, 0.3923, 0.4530, 0.5526],
        [0.6244, 0.4464, 0.5806, 0.3578, 0.4572, 0.5682],
        [0.6019, 0.4468, 0.5688, 0.3919, 0.4585, 0.5638],
        [0.6246, 0.4709, 0.5685, 0.3546, 0.4485, 0.5632],
        [0.6160, 0.4416, 0.5597, 0.3859, 0.4448, 0.5639],
  

        [1.0000, 0.4600, 1.0000, 0.3745, 0.4586, 1.0000]])
classification acc: 0.140625


In [ ]:
#算算整個分類模型以及裡頭的簡單分類器有多少參數
def get_learnable_params(module):
	return [p for p in module.parameters() if p.requires_grad]
	 
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")


整個分類模型的參數量：108314886
線性分類器的參數量：4614



In [ ]:
###---4. 訓練該下游任務模型
# train mode
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1.0e-2)


EPOCHS = 60  # 幸運數字
for epoch in range(EPOCHS):
	
	running_loss = 0.0
	for data in trainloader:
		
		tokens_tensors, segments_tensors, \
		masks_tensors, labels = [t.to(device) for t in data]

		# 將參數梯度歸零
		optimizer.zero_grad()
		
		# forward pass
		outputs = model(input_ids=tokens_tensors, 
						token_type_ids=segments_tensors, 
						attention_mask=masks_tensors, 
						labels=labels)
		#print('outputs:')
		#print(outputs)
		#loss = outputs[0]
		loss = outputs
		# backward
		loss.backward()
		optimizer.step()


		# 紀錄當前 batch loss
		running_loss += loss.item()
		
	# 計算分類準確率
	_, acc = get_predictions(model, trainloader, compute_acc=True)

	print('[epoch %d] loss: %.3f, acc: %.3f' %
		  (epoch + 1, running_loss, acc))

torch.save(model, 'paper_predict.pkl')  # 保存整个网络
torch.save(model.state_dict(), 'paper_predict_params.pkl')   # 只保存网络中的参数 (速度快, 占内存少)